Linear Polymer Melt
===========

insert description

license??

Sources
--------
1. Curro, J.G.; Schweizer K.S.; Integral Equation Theory of Homopolymer
   Melts, Molecular Crystals and Liquid Crystals Incorporating Nonlinear 
   Optics, 180:1, 77-89, doi: 10.1080/00268949008025790
 
  

Global Imports and Setup
----------------------------

In [ ]:
import numpy as np
import holoviews as hv
hv.extension('bokeh')

%load_ext autoreload
%autoreload 2

In [ ]:
%opts Curve Scatter [width=500,height=400] Layout [shared_axes=False] Scatter (size=10,alpha=0.5)
%opts Curve Scatter [fontsize={'xlabel':14,'xlabel':14,'ylabel':14,'ticks':12}]
%opts Overlay [legend_position='bottom_left']
%opts Layout [shared_axes=False]


colors = {}
colors[1.0] = 'blue'
colors[0.8] = 'red'
colors[0.6] = 'green'

ls = {}
ls[1.0] = 'solid'
ls[0.8] = 'dashed'
ls[0.6] = 'dotted'

markers = {}
markers[1.0] = 'o'
markers[0.8] = '^'
markers[0.6] = 'd'

Demo
-----

First we'll load in the data extracted from the literature that we'll be comparing against. 

In [ ]:
gr_compare = []
sk_compare = []
for density in [0.6,0.8,1.0]:
    fname = './data/LinearMelt-Gr-rho{}.csv'.format(density)
    x,y = np.loadtxt(fname,delimiter=',').T
    gr_compare.append([density,x,y])
    
    fname = './data/LinearMelt-Sk-rho{}.csv'.format(density)
    x,y = np.loadtxt(fname,delimiter=',').T
    sk_compare.append([density,x,y])


Next, we'll use typyPRISM to calculate data for a linear polymer melt of of gaussian chains of length 16000, as described in Reference 1. 

In [ ]:
import typyPRISM
from typyPRISM.calculate.prism.structure_factor import structure_factor
from typyPRISM.calculate.prism.pair_correlation import pair_correlation
import numpy as np

sys = typyPRISM.System(['polymer'],kT=1.0)
sys.domain = typyPRISM.Domain(dr=0.005,length=32768)
sys.closure['polymer','polymer'] = typyPRISM.closure.PercusYevick()
sys.potential['polymer','polymer'] = typyPRISM.potential.HardSphere(sigma=1.0)
sys.omega['polymer','polymer'] = typyPRISM.omega.Gaussian(sigma=1.0,length=16000)


gr_results = []
sk_results = []
guess = np.zeros_like(sys.domain.r)
for density in [0.6,0.8,1.0]:
    print('==> Solving for density {}'.format(density))
    sys.density['polymer'] = density
    PRISM = sys.createPRISM()
    result = PRISM.solve(guess)
    guess = np.copy(PRISM.x)
    
    y = structure_factor(PRISM)['polymer','polymer']
    x = sys.domain.k
    sk_results.append([density,x,y])
    
    x = sys.domain.r
    y = pair_correlation(PRISM)['polymer','polymer']
    gr_results.append([density,x,y])
    
    


We start by reproducing Figure 1 of Reference 1.

In [ ]:
%%opts Overlay [legend_position='bottom_right']
from math import sqrt

extents = (0,0,0.6,1.0)

gr_plots = []
for rho,x,y in gr_results:
    Rg = sqrt(16000/6.0)
    label = 'rho={} (typyPRISM)'.format(rho)
    style = {'line_dash':ls[rho],'color':colors[rho]}
    c1 = hv.Curve((x/Rg,y),label=label,extents=extents)(style=style)
    gr_plots.append(c1)
    
for rho,x,y in gr_compare:
    label = 'rho={} (Ref 1)'.format(rho)
    style = {'marker':markers[rho],'color':colors[rho]}
    c1 = hv.Scatter((x,y),label=label,extents=extents)(style=style)
    gr_plots.append(c1)
    
    
hv.Overlay(gr_plots).redim.label(x='r',y='g(r)')

Next, we reproduce Figure 3:

In [ ]:
sk_plots = []
for rho,x,y in sk_results:
    yd = y - 1.0
    label = 'rho={} (typyPRISM)'.format(rho)
    style = {'line_dash':ls[rho],'color':colors[rho]}
    c1 = hv.Curve((x,yd/yd[0]),label=label,extents=(0,0,3,None))(style=style)
    sk_plots.append(c1)
    
for rho,x,y in sk_compare:
    style = {'marker':markers[rho],'color':colors[rho]}
    c1 = hv.Scatter((x,y),label='rho={} (Ref 1)'.format(rho),extents=(0.0,0,3,None))(style=style)
    sk_plots.append(c1)
    
    
hv.Overlay(sk_plots).redim.label(x='k',y='S(k)')

Finally, we reproduce the inset of Figure 3:

In [ ]:
sk0_plots = []
for rho,x,y in sk_results:
    yd = y - 1
    label = 'rho={} (typyPRISM)'.format(rho)
    style = {'line_dash':ls[rho],'color':colors[rho]}
    c1 = hv.Scatter((rho,yd[0]),label=label,extents=(0.4,0,1.2,11))(style=style)
    sk0_plots.append(c1)
    
hv.Overlay(sk0_plots).redim.label(x='rho',y='S(0)')